# Robust News Classification: Main Experiments

This notebook ties together all components of the robust news classification project for the current flow (train on full ISOT, test on held-out files).

## Overview

This notebook demonstrates:
1. **Data Loading & Preprocessing**: Load ISOT training data and clean text
2. **Training**: Train on the full ISOT training set (Fake + True)
3. **Baseline Models**: TF-IDF + LogReg/SVM
4. **Advanced Models**: Sentence embeddings (MiniLM) and Transformer (DistilBERT)
5. **Evaluation**:
   - Fake-only test (`data/test/fake.csv`): false negatives / fake recall
   - Mixed labeled test (`data/test/WELFake_Dataset_sample_10000.csv`): Macro-F1, ROC-AUC, PR-AUC, confusion matrix
6. **Cross-Dataset Transfer**: WELFake evaluation covers the external test without fine-tuning

## Project Goals

- Train on full labeled ISOT data (text only)
- Check false negatives on fake-only held-out data
- Measure balanced metrics on a mixed external set (WELFake)
- Compare TF-IDF baselines with embedding and transformer models

## 1. Setup and Imports

Import all necessary modules from the `src/` directory and configure settings.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

# Add src directory to path for imports
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root / 'src'))

# Import using importlib for files with numeric prefixes
import importlib.util

# Import preprocessing utilities
spec = importlib.util.spec_from_file_location("preprocessing", project_root / "src" / "01_preprocessing.py")
preprocessing = importlib.util.module_from_spec(spec)
spec.loader.exec_module(preprocessing)
sys.modules["preprocessing"] = preprocessing
from preprocessing import load_isot, apply_cleaning, clean_text

# Import baseline models
spec = importlib.util.spec_from_file_location("baseline_models", project_root / "src" / "03_baseline_models.py")
baseline_models = importlib.util.module_from_spec(spec)
spec.loader.exec_module(baseline_models)
sys.modules["baseline_models"] = baseline_models
from baseline_models import build_tfidf, train_logreg, train_svm

# Import evaluation
spec = importlib.util.spec_from_file_location("baseline_eval", project_root / "src" / "04_baseline_eval.py")
baseline_eval = importlib.util.module_from_spec(spec)
spec.loader.exec_module(baseline_eval)
sys.modules["baseline_eval"] = baseline_eval
from baseline_eval import evaluate

print("All imports successful!")
print(f"Project root: {project_root}")

All imports successful!
Project root: /Users/reuben/robust-news-classification/robust-news-classification


## 2. Data Loading and Preprocessing

Load the ISOT dataset (Fake and True news) and apply text cleaning to remove noise and standardize formatting.

In [2]:
# Load ISOT dataset
print("Loading ISOT dataset...")
df = load_isot(
    fake_path='../data/training/Fake.csv',
    real_path='../data/training/True.csv'
)

print(f"\nDataset shape: {df.shape}")
print(f"Columns: {df.columns.tolist()}")
print(f"\nLabel distribution:")
print(df['label'].value_counts())
print(f"\nSubject distribution:")
print(df['subject'].value_counts())

# Apply text cleaning
print("\n" + "="*60)
print("Applying text cleaning...")
df = apply_cleaning(df, text_column='text')
df = apply_cleaning(df, text_column='title')

print(f"\nSample cleaned text (first 200 chars):")
print(df['text_cleaned'].iloc[0][:200])

Loading ISOT dataset...
Loading fake news from ../data/training/Fake.csv...
Loading real news from ../data/training/True.csv...
Loaded 23481 fake articles and 21417 real articles
Total: 44898 articles

Dataset shape: (44898, 6)
Columns: ['title', 'text', 'subject', 'date', 'label', 'source_file']

Label distribution:
label
1    23481
0    21417
Name: count, dtype: int64

Subject distribution:
subject
politicsNews       11272
worldnews          10145
News                9050
politics            6841
left-news           4459
Government News     1570
US_News              783
Middle-east          778
Name: count, dtype: int64

Applying text cleaning...
Applied text cleaning to column 'text'
Created new column 'text_cleaned' with cleaned text
Applied text cleaning to column 'title'
Created new column 'title_cleaned' with cleaned text

Sample cleaned text (first 200 chars):
Donald Trump just couldn t wish all Americans a Happy New Year and leave it at that. Instead, he had to give a shout ou

## 3. Train/Test Setup

Train on the full ISOT training set (Fake + True). Evaluate on two held-out files:
- `data/test/fake.csv` (all fake) to measure false negatives (fake recall)
- `data/test/WELFake_Dataset_sample_10000.csv` (mixed, labeled) for full metrics

In [3]:
print("="*60)
print("Preparing train and held-out test sets")
print("="*60)

# Train on full ISOT training set
X_train_full = df['text_cleaned'].tolist()
y_train_full = df['label'].values

# Fake-only test set: measure false negatives / fake recall
print("\nLoading fake-only test set (all fake)...")
df_fake_test = pd.read_csv('../data/test/fake.csv')
df_fake_test['text_cleaned'] = df_fake_test['text'].apply(clean_text)
X_test_fake = df_fake_test['text_cleaned'].tolist()
# Fake class = 1 (original convention)
y_test_fake = np.ones(len(df_fake_test), dtype=int)

# WELFake mixed test set: full metrics
print("\nLoading WELFake test set (mixed labeled)...")
df_welfake = pd.read_csv('../data/test/WELFake_Dataset_sample_10000.csv')
# WELFake labels are actually 0=real, 1=fake; this matches our convention, so no mapping needed
# If you find labels are reversed, flip with {0:1, 1:0}
df_welfake['text_cleaned'] = df_welfake['text'].apply(clean_text)
X_test_welfake = df_welfake['text_cleaned'].tolist()
y_test_welfake = df_welfake['label'].values

print(f"Train size: {len(X_train_full)}")
print(f"Fake-only test size: {len(X_test_fake)}")
print(f"WELFake test size: {len(X_test_welfake)}")

Preparing train and held-out test sets

Loading fake-only test set (all fake)...

Loading WELFake test set (mixed labeled)...
Train size: 44898
Fake-only test size: 12999
WELFake test size: 1000


## 4. Baseline Models: TF-IDF + Linear Classifiers

Train interpretable baseline models using TF-IDF features with Logistic Regression and Linear SVM.

In [4]:
# Build TF-IDF vectorizer
print("="*60)
print("BASELINE MODELS: TF-IDF + Linear Classifiers")
print("="*60)
vectorizer = build_tfidf(max_features=5000, ngram_range=(1, 2))

# Transform text data (fit once on full training set)
X_train_tfidf = vectorizer.fit_transform(X_train_full)
X_test_fake_tfidf = vectorizer.transform(X_test_fake)
X_test_welfake_tfidf = vectorizer.transform(X_test_welfake)

print(f"\nTF-IDF feature matrix shape: {X_train_tfidf.shape}")

BASELINE MODELS: TF-IDF + Linear Classifiers
Created TF-IDF vectorizer with:
  max_features: 5000
  min_df: 2
  max_df: 0.95
  ngram_range: (1, 2)
  stop_words: english

TF-IDF feature matrix shape: (44898, 5000)


In [5]:
# Train and evaluate Logistic Regression on full train -> WELFake
print("\n" + "="*60)
print("Logistic Regression - Full Train -> WELFake")
print("="*60)
model_lr = train_logreg(X_train_tfidf, y_train_full)
results_lr_welfake = evaluate(model_lr, X_test_welfake_tfidf, y_test_welfake, 
                             model_name="Logistic Regression (WELFake)")


Logistic Regression - Full Train -> WELFake
Training Logistic Regression classifier...
  Training samples: 44898
  C (regularization): 1.0
  max_iter: 1000
  solver: lbfgs
Training completed.
  Training accuracy: 0.9925

Evaluating Logistic Regression (WELFake)
Test set size: 1000 samples
Class distribution: {np.int64(0): np.int64(466), np.int64(1): np.int64(534)}

Metrics                   Value          
----------------------------------------
Accuracy                  0.8450
Precision (macro)         0.8796
Recall (macro)            0.8346
F1-score (macro)          0.8379  <-- PRIMARY METRIC
F1-score (weighted)       0.8402
ROC-AUC                   0.9047  <-- Secondary metric
PR-AUC (Avg Precision)    0.8920  <-- Secondary metric

Confusion Matrix:
                Predicted Real  Predicted Fake 
Actual Real     318             148            
Actual Fake     7               527            

Detailed Classification Report:
              precision    recall  f1-score   support

  

In [6]:
# Train and evaluate Linear SVM on full train -> WELFake
print("\n" + "="*60)
print("Linear SVM - Full Train -> WELFake")
print("="*60)
model_svm = train_svm(X_train_tfidf, y_train_full)
results_svm_welfake = evaluate(model_svm, X_test_welfake_tfidf, y_test_welfake,
                              model_name="Linear SVM (WELFake)")


Linear SVM - Full Train -> WELFake
Training Linear SVM classifier...
  Training samples: 44898
  C (regularization): 1.0
  max_iter: 1000
  dual: False
Training completed.
  Training accuracy: 0.9990

Evaluating Linear SVM (WELFake)
Test set size: 1000 samples
Class distribution: {np.int64(0): np.int64(466), np.int64(1): np.int64(534)}

Metrics                   Value          
----------------------------------------
Accuracy                  0.8360
Precision (macro)         0.8753
Recall (macro)            0.8249
F1-score (macro)          0.8277  <-- PRIMARY METRIC
F1-score (weighted)       0.8302
ROC-AUC                   0.9053  <-- Secondary metric
PR-AUC (Avg Precision)    0.8927  <-- Secondary metric

Confusion Matrix:
                Predicted Real  Predicted Fake 
Actual Real     308             158            
Actual Fake     6               528            

Detailed Classification Report:
              precision    recall  f1-score   support

        Real       0.98      0.

In [7]:
# Fake-only test: false negatives / fake recall (TF-IDF models)
print("\n" + "="*60)
print("Fake-only test set (all fake) - TF-IDF models")
print("="*60)

def fake_only_report(model, X_fake, model_name):
    preds = model.predict(X_fake)
    total = len(preds)
    # Fake class = 1; false negatives are predictions of 0 (real)
    fn = np.sum(preds == 0)
    recall_fake = 1 - fn / total if total else float('nan')
    print(f"{model_name}: total={total}, false_negatives={fn}, fake_recall={recall_fake:.4f}")
    return {"total": int(total), "false_negatives": int(fn), "fake_recall": float(recall_fake)}

fake_only_lr = fake_only_report(model_lr, X_test_fake_tfidf, "LogReg (fake-only)")
fake_only_svm = fake_only_report(model_svm, X_test_fake_tfidf, "Linear SVM (fake-only)")


Fake-only test set (all fake) - TF-IDF models
LogReg (fake-only): total=12999, false_negatives=826, fake_recall=0.9365
Linear SVM (fake-only): total=12999, false_negatives=685, fake_recall=0.9473


## 5. Advanced Models: Sentence Embeddings

Train models using sentence embeddings as features, providing richer semantic representations than TF-IDF.

**Note**: This section requires the `sentence-transformers` library. Uncomment and run if available.

In [8]:
# Import embedding utilities
spec = importlib.util.spec_from_file_location("embeddings_model", project_root / "src" / "05_embeddings_model.py")
embeddings_model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(embeddings_model)
sys.modules["embeddings_model"] = embeddings_model
from embeddings_model import build_embeddings, embed_text, train_embedding_classifier

# Build sentence embedding model
print("="*60)
print("ADVANCED MODELS: Sentence Embeddings")
print("="*60)
embedder = build_embeddings(model_name="all-MiniLM-L6-v2")

# Compute embeddings for full train and tests
print("\nComputing embeddings for full train and test sets...")
emb_train_full = embed_text(embedder, X_train_full)
emb_test_fake_emb = embed_text(embedder, X_test_fake)
emb_test_welfake_emb = embed_text(embedder, X_test_welfake)

print(f"Embedding dimension: {emb_train_full.shape[1]}")

ADVANCED MODELS: Sentence Embeddings
Loading sentence-embedding model: all-MiniLM-L6-v2
Embedding model loaded successfully.

Computing embeddings for full train and test sets...
Encoding 44898 texts into embeddings...


Batches:   0%|          | 0/1404 [00:00<?, ?it/s]

Generated embeddings with shape: (44898, 384)
Encoding 12999 texts into embeddings...


Batches:   0%|          | 0/407 [00:00<?, ?it/s]

Generated embeddings with shape: (12999, 384)
Encoding 1000 texts into embeddings...


Batches:   0%|          | 0/32 [00:00<?, ?it/s]

Generated embeddings with shape: (1000, 384)
Embedding dimension: 384


In [9]:
# Train and evaluate embedding-based classifier -> WELFake
print("\n" + "="*60)
print("Embedding-based Classifier - Full Train -> WELFake")
print("="*60)
model_emb = train_embedding_classifier(emb_train_full, y_train_full)
results_emb_welfake = evaluate(model_emb, emb_test_welfake_emb, y_test_welfake,
                              model_name="Embedding Classifier (WELFake)")


Embedding-based Classifier - Full Train -> WELFake
Training Logistic Regression classifier on embeddings...
  Training samples: 44898
  Embedding dimension: 384
  C (regularization): 1.0
  max_iter: 5000


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

Training completed.
  Training accuracy on embeddings: 0.9617

Evaluating Embedding Classifier (WELFake)
Test set size: 1000 samples
Class distribution: {np.int64(0): np.int64(466), np.int64(1): np.int64(534)}

Metrics                   Value          
----------------------------------------
Accuracy                  0.8120
Precision (macro)         0.8200
Recall (macro)            0.8059
F1-score (macro)          0.8080  <-- PRIMARY METRIC
F1-score (weighted)       0.8099
ROC-AUC                   0.8913  <-- Secondary metric
PR-AUC (Avg Precision)    0.8896  <-- Secondary metric

Confusion Matrix:
                Predicted Real  Predicted Fake 
Actual Real     334             132            
Actual Fake     56              478            

Detailed Classification Report:
              precision    recall  f1-score   support

        Real       0.86      0.72      0.78       466
        Fake       0.78      0.90      0.84       534

    accuracy                           0.81      10

In [10]:
# Fake-only test: false negatives / fake recall (embeddings)
print("\n" + "="*60)
print("Fake-only test set (all fake) - Embedding model")
print("="*60)
emb_preds_fake = model_emb.predict(emb_test_fake_emb)
total_fake = len(emb_preds_fake)
# Fake class = 1; false negatives are predictions of 0 (real)
fn_fake = np.sum(emb_preds_fake == 0)
recall_fake_emb = 1 - fn_fake / total_fake if total_fake else float('nan')
print(f"Embedding model: total={total_fake}, false_negatives={fn_fake}, fake_recall={recall_fake_emb:.4f}")
fake_only_emb = {"total": int(total_fake), "false_negatives": int(fn_fake), "fake_recall": float(recall_fake_emb)}


Fake-only test set (all fake) - Embedding model
Embedding model: total=12999, false_negatives=3235, fake_recall=0.7511


## 5b. Advanced Models: Transformer (DistilBERT)

Fine-tune a DistilBERT transformer on the ISOT training data and evaluate on WELFake.

**Note**: This section requires `transformers` and `torch`. Training takes ~10-20 minutes on CPU, faster on GPU.

In [11]:
# Import transformer utilities
spec = importlib.util.spec_from_file_location("transformer_model", project_root / "src" / "06_transformer_model.py")
transformer_model = importlib.util.module_from_spec(spec)
spec.loader.exec_module(transformer_model)
sys.modules["transformer_model"] = transformer_model
from transformer_model import build_transformer, train_transformer, TransformerSklearnWrapper

print("="*60)
print("ADVANCED MODELS: Transformer (DistilBERT)")
print("="*60)

# Build transformer model and tokenizer
model_transformer, tokenizer = build_transformer(model_name="distilbert-base-uncased")

ADVANCED MODELS: Transformer (DistilBERT)
Loading transformer model 'distilbert-base-uncased' for sequence classification...


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Transformer model and tokenizer loaded successfully.


In [ ]:
# Fine-tune transformer on ISOT training data
# Note: This can take 10-20+ minutes on CPU. Reduce num_train_epochs or use GPU for faster training.
print("\n" + "="*60)
print("Fine-tuning DistilBERT on ISOT data...")
print("="*60)

model_transformer = train_transformer(
    model=model_transformer,
    tokenizer=tokenizer,
    train_texts=X_train_full,
    train_labels=y_train_full,
    output_dir=str(project_root / "models" / "distilbert_isot"),
    num_train_epochs=1,  # Start with 1 epoch for faster iteration; increase to 3 for better results
    per_device_train_batch_size=16,
    learning_rate=5e-5,
    weight_decay=0.01,
    logging_steps=200,
)


Fine-tuning DistilBERT on ISOT data...
Preparing dataset for transformer fine-tuning...
Configuring TrainingArguments...
Initializing Trainer and starting training...


Step,Training Loss
200,0.047400
400,0.008000
600,0.002200
800,0.006300
1000,0.003400
1200,0.000100
1400,0.006400
1600,0.002800
1800,0.008300


In [ ]:
# Wrap transformer for sklearn-compatible evaluation
print("\n" + "="*60)
print("Transformer Evaluation - Full Train -> WELFake")
print("="*60)

transformer_wrapper = TransformerSklearnWrapper(
    model=model_transformer,
    tokenizer=tokenizer,
    max_length=256,
)

# Evaluate on WELFake (pass raw text - wrapper handles tokenization)
results_transformer_welfake = evaluate(
    transformer_wrapper, 
    X_test_welfake,  # Raw text, not embeddings
    y_test_welfake, 
    model_name="DistilBERT Transformer (WELFake)"
)

In [ ]:
# Fake-only test: false negatives / fake recall (transformer)
print("\n" + "="*60)
print("Fake-only test set (all fake) - Transformer model")
print("="*60)

# Note: This may take a few minutes due to the large test set (12999 samples)
# Process in batches to avoid memory issues
batch_size = 64
transformer_preds_fake = []

print(f"Running inference on {len(X_test_fake)} samples in batches of {batch_size}...")
for i in range(0, len(X_test_fake), batch_size):
    batch = X_test_fake[i:i+batch_size]
    preds = transformer_wrapper.predict(batch)
    transformer_preds_fake.extend(preds)
    if (i // batch_size + 1) % 50 == 0:
        print(f"  Processed {min(i+batch_size, len(X_test_fake))}/{len(X_test_fake)} samples...")

transformer_preds_fake = np.array(transformer_preds_fake)
total_fake_transformer = len(transformer_preds_fake)
# Fake class = 1; false negatives are predictions of 0 (real)
fn_fake_transformer = np.sum(transformer_preds_fake == 0)
recall_fake_transformer = 1 - fn_fake_transformer / total_fake_transformer if total_fake_transformer else float('nan')

print(f"Transformer: total={total_fake_transformer}, false_negatives={fn_fake_transformer}, fake_recall={recall_fake_transformer:.4f}")
fake_only_transformer = {"total": int(total_fake_transformer), "false_negatives": int(fn_fake_transformer), "fake_recall": float(recall_fake_transformer)}

## 6. Results Summary

Compare models on WELFake (mixed labeled) and report fake-only recall on `data/test/fake.csv`.

In [ ]:
# Compile results summary
print("="*60)
print("RESULTS SUMMARY")
print("="*60)

results_summary = pd.DataFrame({
    'Model': [
        'Logistic Regression (TF-IDF)',
        'Linear SVM (TF-IDF)',
        'Embedding Classifier',
        'DistilBERT Transformer',
    ],
    'WELFake - Macro-F1': [
        results_lr_welfake['f1_macro'],
        results_svm_welfake['f1_macro'],
        results_emb_welfake['f1_macro'] if 'results_emb_welfake' in locals() else None,
        results_transformer_welfake['f1_macro'] if 'results_transformer_welfake' in locals() else None,
    ],
    'WELFake - ROC-AUC': [
        results_lr_welfake['roc_auc'],
        results_svm_welfake['roc_auc'],
        results_emb_welfake['roc_auc'] if 'results_emb_welfake' in locals() else None,
        results_transformer_welfake['roc_auc'] if 'results_transformer_welfake' in locals() else None,
    ],
    'WELFake - PR-AUC': [
        results_lr_welfake['pr_auc'],
        results_svm_welfake['pr_auc'],
        results_emb_welfake['pr_auc'] if 'results_emb_welfake' in locals() else None,
        results_transformer_welfake['pr_auc'] if 'results_transformer_welfake' in locals() else None,
    ],
    'Fake-only recall': [
        fake_only_lr['fake_recall'],
        fake_only_svm['fake_recall'],
        fake_only_emb['fake_recall'] if 'fake_only_emb' in locals() else None,
        fake_only_transformer['fake_recall'] if 'fake_only_transformer' in locals() else None,
    ],
})

print("\nModel Performance Comparison:")
print(results_summary.to_string(index=False))

## 7. Cross-Dataset Transfer Evaluation

Handled above by running the ISOT-trained models on WELFake (zero-shot, no fine-tuning). No additional code needed here.

(Reference) Cross-dataset transfer summary: see WELFake results above; no further actions here.

## 8. Conclusions and Discussion

### Key Findings:

1. **Model Performance**: Compare baseline (TF-IDF) vs. advanced (embeddings, transformers) approaches
2. **Robustness**: Assess performance drop from random splits to topic-holdout splits
3. **Generalization**: Evaluate cross-dataset transfer performance on WELFake

### Interpretation:

- **Macro-F1** serves as the primary metric to balance performance across classes
- **Topic-holdout splits** reveal whether models rely on topic-specific shortcuts
- **Cross-dataset evaluation** tests real-world applicability beyond ISOT

### Next Steps:

- Fine-tune hyperparameters for optimal performance
- Analyze failure cases and model interpretability
- Expand cross-dataset evaluation to additional external datasets
- Add transformer model evaluation (requires additional setup)